# ***TS-ILS Hyper-heuristic***

***TS-ILS Hyper-heuristic : Thompson Sampling Iterated Local Search***

L'Hyper-Heuristiques TS-ILS et l'une des méthodes efficaces pour résoudre de nombreux problèmes d'optimisation tels que : PVC, BP, SAT, ...
Elle respose sur différentes démarche qui différent en terme d'exploitation des heuristiques de bas niveau, le haut niveau quand à lui restent fidèle et apt à répondre à différentes instances d'un même problème et différents problèmes, ici nous avions implémenté un modèle simple d'une TS-ILS dans le principe est le suivant :


1.   ***Initialisation :***
- **Ensemble d'heuristiques de perturbation.**
- **Force de perturbation :** Une force de perturbation plus élevée entraîne des perturbations plus importantes, ce qui peut permettre à l'algorithme d'explorer un espace de recherche plus large. À l'inverse, une force de perturbation plus faible entraînera des perturbations plus légères, ce qui peut permettre une exploration plus locale et concentrée de l'espace de recherche.
- **Alpha Vector :** Plus une heuristique est choisie avec succès, plus son paramètre alpha associé sera élevé.
- **Béta Vector :** Plus une heuristique est choisie mais n'améliore pas la solution, plus son paramètre beta associé sera élevé.  
2.   ***Génération d'une solution initiale S0 :*** Aléatoirement.
3.   ***Itéartions :*** jusqu'à max_iter initialisé au début.
4.   ***Pour chaque itéartion :*** nous avions ces étapes.
- Calculer l'utilité des heuristiques puis sélectionner une configuration meilleure c'est à dire une heuristique plus efficace.
- Appliquer l'heuristique choisi en ayant en entrée la solution courante(S0).
- **Recherche locale (2-opt) :** Pousser encore la solution en appliquant une amélioration par recherche locale.
- **Acceptation de la solution améliorée (S_double_prime) :** Si la solution améliorée (S_double_prime) a un coût inférieur à celui de la solution actuelle (S0), alors elle est acceptée comme la nouvelle solution courante (S0). Cela garantit que l'algorithme progresse vers des solutions de meilleure qualité.
- **Mise à jour de la meilleure solution (Sb) :** Si la solution améliorée (S_double_prime) a un coût inférieur à celui de la meilleure solution connue (Sb), alors elle devient la nouvelle meilleure solution (Sb). Le coût de la meilleure solution (Sb_cost) est également mis à jour en conséquence.
- **Mise à jour des paramètres alpha et béta :**
   - Si la solution améliorée (S_double_prime) a un coût inférieur à celui de la meilleure solution courante (Sb), alors l'heuristique qui a été sélectionnée pour cette amélioration reçoit un point positif dans le vecteur alpha.
   - Sinon, la même heuristique reçoit un point positif dans le vecteur beta.
   - Cette mise à jour des vecteurs alpha et beta permet de mesurer l'efficacité des différentes heuristiques au fil du temps.
- **Mise à jour de la force de perturbation :**
   - À chaque itération, la force de perturbation est ajustée en fonction de la logique définie dans la méthode `update_ls`.
5. ***Itération suivante :*** Une fois toutes ces étapes terminées, l'itération est incrémentée et le processus se répète jusqu'à ce que le nombre maximal d'itérations soit atteint.

6. ***Retour de la meilleure solution :*** À la fin de toutes les itérations, la meilleure solution (Sb) et son coût associé (Sb_cost) sont retournés.



***Heuristiques de bas niveau***

1. `perturb` : Cette heuristique échange simplement deux éléments choisis aléatoirement dans la solution.

2. `inversion_subsequence` : Cette heuristique inverse l'ordre des éléments d'une sous-séquence de la solution, où les indices de début et de fin de la sous-séquence sont choisis aléatoirement.

3. `random_swap` : Contrairement aux deux premières, random_swap divise la solution en plusieurs sous-séquences de longueurs aléatoires, puis échange aléatoirement un élément de chaque sous-séquence. Cela garantit que les éléments échangés sont plus éloignés dans la solution, introduisant ainsi des perturbations plus étendues.

***Tests sur les matrices de distances***

```
Instance:(Cost TSILS)/(Exact)
eil101 : 677/629
gil262 : 2522/2378
```


***Avantages :***

1. Fournit des solutions proches des solutions exactes.
2. Stratégie de haut niveau basée sur l'historique(Paramètre Apha et Béta) ce qui permet à l'algorithme de s'adapter à différentes instances de problèmes grâce à un niveau d'abstraction élevé.
3. Encouragement de l'intensification par recherche locale.
4. Encouragement de la diversification par perturbation(Heuristiques de perturbation).

***Inconvénients :***
1. Peut prendre du temps si le nombre d'itérations est élevé et/ou si le nombre de villes (ou de variables) dans le problème est élevé.

***Implémentation***

In [ ]:
import numpy as np
import random

# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

# Génération d'une solution initiale
def generate_initial_solution(num_villes):
    solution = list(range(num_villes))
    random.shuffle(solution)
    return solution

# Perturbation d'une solution
def perturb(solution, perturbation_strength):
    new_solution = solution.copy()
    for _ in range(perturbation_strength):
        i, j = random.sample(range(len(solution)), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution

def inversion_subsequence(solution, perturbation_strength):
    new_solution = solution.copy()
    for _ in range(perturbation_strength):
        i, j = sorted(random.sample(range(len(solution)), 2))
        new_solution[i:j+1] = new_solution[i:j+1][::-1]
    return new_solution

def random_swap(solution, perturbation_strength):
    new_solution = solution.copy()
    num_swaps = min(perturbation_strength, len(solution) // 2)
    for _ in range(num_swaps):
        subsequence_length = random.randint(2, len(solution) // 2)
        start_index = random.randint(0, len(solution) - subsequence_length)
        end_index = start_index + subsequence_length
        i, j = random.sample(range(start_index, end_index), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution


# Recherche locale simple (2-opt)
def local_search(solution, matrice_distances):
    n = len(solution)
    improved = True
    while improved:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if j - i == 1: continue
                new_solution = solution[:i+1] + solution[i+1:j+1][::-1] + solution[j+1:]
                if calculer_cout(new_solution, matrice_distances) < calculer_cout(solution, matrice_distances):
                    solution = new_solution
                    improved = True
    return solution

class TSILS_HyperHeuristic:
    def __init__(self, heuristics, max_iter, perturbation_strength):
        self.heuristics = heuristics                             # Heuristiques à utiliser
        self.max_iter = max_iter                                 # Nombre Max d'itérations
        self.perturbation_strength = perturbation_strength       # Force de perturbation
        self.alpha = [0] * len(heuristics)
        self.beta = [0] * len(heuristics)

    def run(self, matrice_distances):
        num_villes = len(matrice_distances)
        S0 = generate_initial_solution(num_villes)
        Sb = S0
        Sb_cost = calculer_cout(Sb, matrice_distances)

        iteration = 0
        while iteration < self.max_iter:
            Phi = self.generate_utility_values()
            i = self.select_config(Phi)
            selected_heuristic = self.heuristics[i]

            if selected_heuristic == perturb:
                S_prime = perturb(S0, self.perturbation_strength)
            elif selected_heuristic == inversion_subsequence:
                S_prime = inversion_subsequence(S0, self.perturbation_strength)
            S_double_prime = local_search(S_prime, matrice_distances)
            S_double_prime_cost = calculer_cout(S_double_prime, matrice_distances)

            if self.accept_solution(S_double_prime_cost, calculer_cout(S0, matrice_distances)):
                S0 = S_double_prime

            if S_double_prime_cost < Sb_cost:
                Sb = S_double_prime
                Sb_cost = S_double_prime_cost
                self.alpha[i] += 1
                self.update_ls()
                self.update_param()
            else:
                self.beta[i] += 1

            self.update_llh()
            iteration += 1

        return Sb, Sb_cost

    def generate_utility_values(self):
        # Utilisation de l'efficacité historique des heuristiques pour générer les valeurs utilitaires
        return [self.alpha[i] - self.beta[i] for i in range(len(self.heuristics))]

    def select_config(self, Phi):
        # Sélectionne l'heuristique avec la valeur utilitaire maximale
        return np.argmax(Phi)

    def accept_solution(self, new_cost, current_cost):
        # Accepte la nouvelle solution si elle est meilleure ou avec une faible probabilité
        return new_cost < current_cost or random.uniform(0, 1) < 0.05

    def update_ls(self):
        # Réinitialiser les paramètres pour éviter la stagnation
        self.perturbation_strength = max(1, self.perturbation_strength - 1)

    def update_param(self):
        # Ajuste les paramètres alpha et beta pour donner plus d'importance aux heuristiques performantes
        total_alpha = sum(self.alpha)
        total_beta = sum(self.beta)
        if total_alpha > 0 and total_beta > 0:
            self.alpha = [a / total_alpha for a in self.alpha]
            self.beta = [b / total_beta for b in self.beta]

    def update_llh(self):
        # Augmente la force de perturbation pour explorer d'autres solutions
        self.perturbation_strength += 1

heuristics = [perturb, inversion_subsequence, random_swap]  # Ajoutez d'autres heuristiques si nécessaire
max_iter = 10
perturbation_strength = 2

hyper_heuristic = TSILS_HyperHeuristic(heuristics, max_iter, perturbation_strength)
best_solution, best_cost = hyper_heuristic.run(distance_matrix)

print("Meilleure solution trouvée:", best_solution)
print("Coût de la meilleure solution:", best_cost)


Meilleure solution trouvée: [89, 60, 220, 128, 129, 48, 123, 52, 227, 64, 205, 82, 35, 219, 149, 182, 57, 229, 169, 1, 252, 192, 100, 113, 147, 25, 8, 152, 247, 188, 105, 120, 88, 166, 12, 202, 56, 213, 10, 234, 240, 224, 222, 42, 81, 156, 248, 61, 236, 176, 68, 145, 134, 83, 15, 242, 51, 101, 38, 121, 37, 241, 55, 255, 9, 130, 237, 41, 196, 193, 79, 112, 102, 65, 167, 46, 54, 174, 258, 16, 23, 39, 126, 146, 244, 200, 77, 31, 66, 50, 133, 158, 157, 59, 109, 108, 19, 28, 226, 40, 209, 177, 91, 104, 131, 96, 225, 135, 175, 232, 14, 218, 74, 161, 58, 170, 180, 168, 5, 36, 164, 221, 84, 251, 254, 110, 235, 22, 47, 195, 173, 97, 116, 239, 106, 13, 142, 127, 119, 21, 43, 198, 250, 207, 151, 199, 62, 179, 215, 45, 178, 160, 92, 186, 44, 211, 76, 90, 49, 87, 246, 150, 93, 85, 148, 118, 217, 183, 191, 143, 238, 34, 259, 155, 27, 181, 94, 33, 86, 187, 257, 3, 80, 185, 20, 137, 99, 144, 114, 153, 75, 154, 107, 243, 69, 98, 208, 253, 233, 67, 141, 231, 230, 201, 138, 0, 228, 70, 30, 204, 32, 18, 1